In [1]:
class Printer:
    def __init__(self):
        pass

    def print(self, value):
        print(value, end='')


class MockPrinter(Printer):
    def __init__(self):
        super().__init__()
        self.saved_output = ""

    def print(self, value):
        if isinstance(value, int):
            self.saved_output += str(value)
        elif isinstance(value, str):
            self.saved_output += value
        else:
            raise TypeError("Unsupported value type for printing")

    def get_output(self):
        return self.saved_output

In [4]:
import unittest

class PrinterTestCase(unittest.TestCase):
    def setUp(self):
        self.mock_printer = MockPrinter()

    def tearDown(self):
        pass

    def test_print_string(self):
        self.mock_printer.print("hello")
        self.mock_printer.print("hello\n")
        expected = "hellohello\n"
        assert expected == self.mock_printer.get_output()

    def test_print_int(self):
        self.mock_printer.print(1234)
        expected = str(1234)
        assert expected == self.mock_printer.get_output()


unittest.main(argv=[''], verbosity=2, exit=False)

test_print_int (__main__.PrinterTestCase) ... ok
test_print_string (__main__.PrinterTestCase) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.013s

OK


In [5]:
class CircularBuffer:
    CAPACITY = 5

    def __init__(self, capacity=CAPACITY):
        self.index = 0
        self.outdex = 0
        self.capacity = capacity
        self.buffer = [0] * self.capacity
        self.empty = True
        self.full = False

    def is_empty(self):
        return self.empty

    def is_full(self):
        return self.full

    def put(self, i):
        self.empty = False
        self.buffer[self.index] = i
        self.index = self.next(self.index)
        if self.full:
            self.outdex = self.next(self.outdex)
        elif self.index == self.outdex:
            self.full = True

    def get(self):
        result = -1
        self.full = False

        if not self.empty:
            result = self.buffer[self.outdex]
            self.outdex = self.next(self.outdex)
            if self.outdex == self.index:
                self.empty = True

        return result

    def get_capacity(self):
        return self.capacity

    def next(self, i):
        if i + 1 >= self.capacity:
            return 0
        return i + 1

    def print(self, printer):
        printer.print("Circular buffer content:\n<")

        print_index = self.outdex
        count = self.index - self.outdex

        if not self.empty and (self.index <= self.outdex):
            count = self.capacity - (self.outdex - self.index)

        for i in range(count):
            printer.print(self.buffer[print_index])
            print_index = self.next(print_index)
            if i + 1 != count:
                printer.print(", ")

        printer.print(">\n")

In [9]:
class CircularBufferTestCase(unittest.TestCase):

    def fill_the_queue(self, seed, how_many):
        for i in range(how_many):
            self.buffer.put(seed + i)

    def remove_from_queue(self, how_many):
        for i in range(how_many):
            self.buffer.get()

    def setUp(self):
        self.mock_printer = MockPrinter()
        self.buffer = CircularBuffer()

    def tearDown(self):
        pass

    def test_empty_after_creation(self):
        self.assertTrue(self.buffer.is_empty())

    def test_not_empty(self):
        self.buffer.put(10046)
        self.assertFalse(self.buffer.is_empty())

    def test_not_empty_then_empty(self):
        self.buffer.put(4567)
        self.assertFalse(self.buffer.is_empty())
        self.buffer.get()
        self.assertTrue(self.buffer.is_empty())

    def test_get_put_one_value(self):
        self.buffer.put(4567)
        self.assertEqual(4567, self.buffer.get())

    def test_get_put_a_few(self):
        self.buffer.put(1)
        self.buffer.put(2)
        self.buffer.put(3)
        self.assertEqual(1, self.buffer.get())
        self.assertEqual(2, self.buffer.get())
        self.assertEqual(3, self.buffer.get())

    def test_capacity(self):
        buffer_two = CircularBuffer(2)
        self.assertEqual(2, buffer_two.get_capacity())

    def test_is_full(self):
        self.fill_the_queue(0, self.buffer.get_capacity())
        self.assertTrue(self.buffer.is_full())

    def test_empty_to_full_to_empty(self):
        self.fill_the_queue(100, self.buffer.get_capacity())
        self.assertTrue(self.buffer.is_full())
        self.remove_from_queue(self.buffer.get_capacity())
        self.assertTrue(self.buffer.is_empty())

    def test_wrap_around(self):
        self.fill_the_queue(100, self.buffer.get_capacity())
        self.assertTrue(self.buffer.is_full())
        self.assertEqual(100, self.buffer.get())
        self.assertFalse(self.buffer.is_full())
        self.buffer.put(1000)
        self.assertTrue(self.buffer.is_full())

        self.remove_from_queue(self.buffer.get_capacity() - 1)
        self.assertEqual(1000, self.buffer.get())
        self.assertTrue(self.buffer.is_empty())

    def test_put_to_full(self):
        current_capacity = self.buffer.get_capacity()
        self.fill_the_queue(900, current_capacity)
        self.buffer.put(9999)

        for i in range(current_capacity - 1):
            self.assertEqual(i + 900 + 1, self.buffer.get())

        self.assertEqual(9999, self.buffer.get())
        self.assertTrue(self.buffer.is_empty())

    def test_get_from_empty(self):
        self.assertEqual(-1, self.buffer.get())
        self.assertTrue(self.buffer.is_empty())

# using mock object
    def test_print_empty(self):
        my_printer = MockPrinter()
        self.buffer.print(my_printer)
        self.assertEqual("Circular buffer content:\n<>\n",
            my_printer.get_output())

    def test_print_after_one_put(self):
        my_printer = MockPrinter()
        self.buffer.put(1)
        self.buffer.print(my_printer)
        self.assertEqual("Circular buffer content:\n<1>\n",
            my_printer.get_output())

    def test_print_not_yet_wrapped_or_full(self):
        my_printer = MockPrinter()
        self.buffer.put(1)
        self.buffer.put(2)
        self.buffer.put(3)
        self.buffer.print(my_printer)
        self.assertEqual("Circular buffer content:\n<1, 2, 3>\n",
            my_printer.get_output())

    def test_print_not_yet_wrapped_and_is_full(self):
        my_printer = MockPrinter()
        self.fill_the_queue(200, self.buffer.get_capacity())
        self.buffer.print(my_printer)
        self.assertEqual("Circular buffer content:\n<200, 201, 202, 203, 204>\n",
            my_printer.get_output())

    def test_print_wrapped_and_is_full_oldest_to_newest(self):
        my_printer = MockPrinter()
        self.fill_the_queue(200, self.buffer.get_capacity())
        self.buffer.get()
        self.buffer.put(999)
        self.buffer.print(my_printer)
        self.assertEqual("Circular buffer content:\n<201, 202, 203, 204, 999>\n",
            my_printer.get_output())

    def test_print_wrapped_and_full_overwrite_oldest(self):
        my_printer = MockPrinter()
        self.fill_the_queue(200, self.buffer.get_capacity())
        self.buffer.put(9999)
        self.buffer.print(my_printer)
        self.assertEqual("Circular buffer content:\n<201, 202, 203, 204, 9999>\n",
            my_printer.get_output())

    def test_print_boundfary(self):
        my_printer = MockPrinter()
        self.fill_the_queue(200, self.buffer.get_capacity())
        self.remove_from_queue(self.buffer.get_capacity() - 2)
        self.buffer.put(888)
        self.fill_the_queue(300, self.buffer.get_capacity() - 1)
        self.buffer.print(my_printer)
        self.assertEqual("Circular buffer content:\n<888, 300, 301, 302, 303>\n",
            my_printer.get_output())

    def test_fill_empty_then_print(self):
        my_printer = MockPrinter()
        self.fill_the_queue(200, self.buffer.get_capacity())
        self.remove_from_queue(self.buffer.get_capacity())
        self.buffer.print(my_printer)
        self.assertEqual("Circular buffer content:\n<>\n",
            my_printer.get_output())


unittest.main(argv=[''], verbosity=2, exit=False)

test_capacity (__main__.CircularBufferTestCase) ... ok
test_empty_after_creation (__main__.CircularBufferTestCase) ... ok
test_empty_to_full_to_empty (__main__.CircularBufferTestCase) ... ok
test_fill_empty_then_print (__main__.CircularBufferTestCase) ... ok
test_get_from_empty (__main__.CircularBufferTestCase) ... ok
test_get_put_a_few (__main__.CircularBufferTestCase) ... ok
test_get_put_one_value (__main__.CircularBufferTestCase) ... ok
test_is_full (__main__.CircularBufferTestCase) ... ok
test_not_empty (__main__.CircularBufferTestCase) ... ok
test_not_empty_then_empty (__main__.CircularBufferTestCase) ... ok
test_print_after_one_put (__main__.CircularBufferTestCase) ... ok
test_print_boundfary (__main__.CircularBufferTestCase) ... ok
test_print_empty (__main__.CircularBufferTestCase) ... ok
test_print_not_yet_wrapped_and_is_full (__main__.CircularBufferTestCase) ... ok
test_print_not_yet_wrapped_or_full (__main__.CircularBufferTestCase) ... ok
test_print_wrapped_and_full_overwrite